# GBDT算法

GBDT（Gradient Boosting Decision Tree），全名叫梯度提升决策树，是一种迭代的决策树算法，又叫 MART（Multiple Additive Regression Tree），它通过构造一组弱的学习器（树），并把多颗决策树的结果累加起来作为最终的预测输出。
该算法将决策树与集成思想进行了有效的结合。

>参考
> *  [图解机器学习 | GBDT模型详解](https://www.showmeai.tech/article-detail/193)

## Boosting核心思想
Boosting方法训练基分类器时采用串行的方式，各个基分类器之间有依赖。它的基本思路是将基分类器层层叠加，每一层在训练的时候，对前一层基分类器分错的样本，给予更高的权重。测试时，根据各层分类器的结果的加权得到最终结果

* Boosting  串行-将基分类器层层叠加
* Bagging   并行-各基分类器之间无强依赖

## GBDT 原理

1. 所有弱分类器的结果相加等于预测值。
2. 每次都以当前预测为基准，下一个弱分类器去拟合误差函数对预测值的残差（预测值与真实值之间的误差）。
3. GBDT的弱分类器使用的是树模型。

![](https://img-blog.csdnimg.cn/img_convert/8b35092a9eeca61fba14a1decd236d8b.png)

### GBDT与负梯度近似残差
回归任务下，GBDT在每一轮的迭代时对每个样本都会有一个预测值，此时的损失函数为均方差损失函数：

$$ l(y_i,\hat y_i) = 1/2 (y_i - \hat y_i) ^2 $$

损失函数的负梯度计算如下：

$$ - \frac{\partial l(y_i,\hat y_i)}{\hat y_i} = (y_i - \hat y_i) $$

1）优点

 * 预测阶段，因为每棵树的结构都已确定，可并行化计算，计算速度快。
 * 适用稠密数据，泛化能力和表达能力都不错，数据科学竞赛榜首常见模型。
 * 可解释性不错，鲁棒性亦可，能够自动发现特征间的高阶关系。

2）缺点

 * GBDT 在高维稀疏的数据集上，效率较差，且效果表现不如 SVM 或神经网络。
 * 适合数值型特征，在 NLP 或文本特征上表现弱。
 * 训练过程无法并行，工程加速只能体现在单颗树构建过程中。

### 随机森林 vs GBDT

1）相同点

* 都是集成模型，由多棵树组构成，最终的结果都是由多棵树一起决定。
* RF 和 GBDT 在使用 CART 树时，可以是分类树或者回归树。

2）不同点

 * 训练过程中，随机森林的树可以并行生成，而 GBDT 只能串行生成
 * 随机森林的结果是多数表决表决的，而 GBDT 则是多棵树累加之。
 * 随机森林对异常值不敏感，而 GBDT 对异常值比较敏感。
 * 随机森林降低模型的方差，而 GBDT 是降低模型的偏差。

使用 sklearn 来对数据拟合和可视化

In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
X = np.arange(1, 11).reshape(-1, 1)
y = np.array([5.16, 4.73, 5.95, 6.42, 6.88, 7.15, 8.95, 8.71, 9.50, 9.15])
gbdt = GradientBoostingRegressor(max_depth=4, criterion ='squared_error').fit(X, y)

from IPython.display import Image
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz
## 拟合训练5棵树
sub_tree = gbdt.estimators_[4, 0]
dot_data = export_graphviz(sub_tree, out_file=None, filled=True, rounded=True, special_characters=True, precision=2)
graph =  graph_from_dot_data(dot_data)
# Image(graph.create_png())
